In [ ]:
import requests
import sys
import os
import pandas as pd
import time
import logging
import shutil

In [ ]:
# Zona horaria de Perú
os.environ['TZ'] = 'America/Lima'
time.tzset()  # Solo funciona en sistemas tipo Unix (como Colab o Linux)

In [ ]:
# Reset y configurar logging para que funcione en Colab / Jupyter
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)

In [ ]:
# Constantes
GITHUB_ROOT_PATH = 'https://github.com/AngelTintaya/datasets/raw/refs/heads/main'
ROOT_DIR = '/content/drive/MyDrive/Datasets/datalake'
OUTPUT_DIR = os.path.join(ROOT_DIR, 'datalake_outputs')

In [ ]:
def get_config_file(file_name: str) -> dict[str:str]:
    """
    """
    GITHUB_FILE_PATH = os.path.join(GITHUB_ROOT_PATH, file_name) #(1) Ruta del archivo en github
    OUTPUT_FILE_PATH = os.path.join(OUTPUT_DIR, file_name) #(2)

    return {
        'GITHUB_FILE': file_name,
        'GITHUB_FILE_PATH': GITHUB_FILE_PATH,
        'OUTPUT_FILE_PATH': OUTPUT_FILE_PATH
    }

In [ ]:
def validar_existencia_archivo(path: str) -> bool:
    """
    """
    response = requests.head(path, allow_redirects=True) #(3) revisa existencia del archivo en github
    if response.status_code != 200:
        logging.warning(f"El archivo no existe en el repositorio: {path}") #(4)
        return False
    return True

In [ ]:
def limpiar_directorio(path: str) -> None:
    """
    """
    if not os.path.exists(path): #(5)
        os.makedirs(path) #(6)
        return

    for item in os.listdir(path): #(7)
        ruta = os.path.join(path, item) #(8)
        if os.path.isfile(ruta): #(9)
            os.remove(ruta) #(10)
        elif os.path.isdir(ruta): #(11)
            shutil.rmtree(ruta) #(12)
    logging.info(f"Directorio limpiado: {path}")

In [ ]:
def leer_y_guardar_csv_en_datalake(file_path: str, output_path: str) -> None:
    """
    """
    try:
        df_csv = pd.read_csv(file_path) #(13)
        df_csv.to_csv(output_path, index=False) #(14)
        logging.info(f"Guardado exitosamente: {output_path}")
    except Exception as e: #(15)
        logging.error(f"Error al procesar archivo {file_path}: {e}")
        raise

In [ ]:
def procesar_archivo_desde_github(file_name: str = 'dummy.csv') -> None:
    """
    """
    logging.info(f"{'='*20} PROCESANDO: {file_name} {'='*20}")

    config_file = get_config_file(file_name) #(16)
    GITHUB_FILE_PATH = config_file['GITHUB_FILE_PATH'] #(17)
    OUTPUT_FILE_PATH = config_file['OUTPUT_FILE_PATH'] #(18)

    # Crear directorio si no existe
    os.makedirs(OUTPUT_DIR, exist_ok=True) #(19)

    if not validar_existencia_archivo(GITHUB_FILE_PATH):
        logging.warning(f"Archivo no encontrado: {GITHUB_FILE_PATH}")
        return

    leer_y_guardar_csv_en_datalake(GITHUB_FILE_PATH, OUTPUT_FILE_PATH) #(20)

    logging.info(f"{'='*20} FINALIZADO: {file_name} {'='*20}")

In [ ]:
def main():
    """
    """
    limpiar_directorio(OUTPUT_DIR) #(21)
    time.sleep(5)  # Solo para demostración visual, puede eliminarse en producción

    archivos = [
        'netflix_titles.csv',
        'netflix_titles_directors.csv',
        'netflix_titles_countries.csv',
        'netflix_titles_cast.csv',
        'netflix_titles_category.csv'
    ] #(22)

    for archivo in archivos: #(23)
        procesar_archivo_desde_github(archivo) #(24)

In [ ]:
# Sólo se ejecuta si el archivo se ejecuta directamente
if __name__ == "__main__": #(25)
    main()

2025-09-19 19:23:58,590 - INFO - Directorio limpiado: /content/drive/MyDrive/Datasets/datalake/datalake_outputs
2025-09-19 19:24:03,592 - INFO - ==================== PROCESANDO: netflix_titles.csv ====================
2025-09-19 19:24:05,145 - INFO - Guardado exitosamente: /content/drive/MyDrive/Datasets/datalake/datalake_outputs/netflix_titles.csv
2025-09-19 19:24:05,149 - INFO - ==================== FINALIZADO: netflix_titles.csv ====================
2025-09-19 19:24:05,150 - INFO - ==================== PROCESANDO: netflix_titles_directors.csv ====================
2025-09-19 19:24:05,731 - INFO - Guardado exitosamente: /content/drive/MyDrive/Datasets/datalake/datalake_outputs/netflix_titles_directors.csv
2025-09-19 19:24:05,735 - INFO - ==================== FINALIZADO: netflix_titles_directors.csv ====================
2025-09-19 19:24:05,737 - INFO - ==================== PROCESANDO: netflix_titles_countries.csv ====================
2025-09-19 19:24:06,250 - INFO - Guardado exitosamen